# Michelin Restaurant Dataset: ImgGeneration (Shutterstock)

Text-to-img models:
- Shutterstock: endpoint already up and running on Databricks but it is not compatible with the resolution parameter to lower it
- stable-diffusion-2-1: available on [Huggingface](https://huggingface.co/stabilityai/stable-diffusion-2-1) and it is mentioned in this [doc](https://docs.google.com/document/d/1GFZzo8paONRC9YYM-nwu1z1DXEqyOqh5ziiSe9kS06Q/edit)

In [0]:
%pip install diffusers
%pip install torch==1.12.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

In [0]:
dbutils.library.restartPython()

In [0]:
import pandas as pd
import os
from pyspark import SparkFiles
from pyspark.sql.functions import *

import requests
import base64
import io
from PIL import Image
from IPython.display import display

### Catalog, Schema Set up

In [0]:
catalog_ = os.getenv('CATALOG_NAME')
schema_ = os.getenv('SCHEMA_NAME')
spark.sql("USE CATALOG "+catalog_)
spark.sql("USE SCHEMA "+schema_)

### Check if secret scope is set up to reach endpoints

In [0]:
scope_name_ = 'michelin_scope'
secret_name_ = 'pat_ga'

In [0]:
# Check scope
existing_scopes = [scope.name for scope in dbutils.secrets.listScopes()]
if scope_name_ in existing_scopes:
    print("Secret scope exists!")
else:
    print("Secret scope doesn't exist, create it via CLI!")

# Check secret
existing_secrets = [secret.key for secret in dbutils.secrets.list(scope_name_)]
if secret_name_ in existing_secrets:
    print("Secret exists!")
else:
    print("Secret doesn't exist, create it via CLI!")

### Read Silver Data

In [0]:
silver_df = spark.sql("SELECT * FROM silver_data")
display(silver_df)

### Create the model

In [0]:
import torch
from diffusers import StableDiffusionPipeline

In [0]:
model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")